In [33]:
# Bibliotecas utlizadas
import pandas as pd
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

In [34]:
# Carregar os dados do MovieLens
dados_avaliacoes = pd.read_csv('ml-100k/u.data', sep='\t', header=None, names=['id_usuario', 'id_filme', 'avaliacao', 'timestamp'])
dados_filmes = pd.read_csv('ml-100k/u.item', sep='|', encoding='ISO-8859-1', header=None, usecols=[0, 1], names=['id_filme', 'titulo_filme'])


In [35]:
# Gêneros aleatórios para cada filme
generos_possiveis = ['Ação', 'Aventura', 'Drama', 'Comédia', 'Terror', 'Romance', 'Sci-Fi', 'Documentário']
dados_filmes['genero'] = [random.choice(generos_possiveis) for _ in range(len(dados_filmes))]

In [36]:
# Junção de avaliações e filmes
dados = pd.merge(dados_avaliacoes, dados_filmes, on='id_filme')

In [37]:
# Remover duplicatas
dados = dados.groupby(['id_usuario', 'id_filme', 'titulo_filme'], as_index=False).agg({'avaliacao': 'mean'})

In [38]:
# Matriz usuário-filme
matriz_usuario_filme = dados.pivot_table(index='id_usuario', columns='titulo_filme', values='avaliacao', fill_value=0)

In [39]:
# SVD (reduz dimensionalidade da matriz)
svd = TruncatedSVD(n_components=50)  # 50 componentes
matriz_reduzida = svd.fit_transform(matriz_usuario_filme)

In [40]:
# Similaridade entre usuários usando a similaridade do cosseno
similaridade_usuarios = cosine_similarity(matriz_reduzida)

In [41]:
# Vetorização TF-IDF para gêneros de filmes
vetorizador_tfidf = TfidfVectorizer()
matriz_tfidf_generos = vetorizador_tfidf.fit_transform(dados_filmes['genero'])

In [42]:
# Similaridade entre filmes usando a similaridade do cosseno
similaridade_filmes = cosine_similarity(matriz_tfidf_generos)

In [43]:
# Função para recomendar filmes
def recomendar_filmes_usuario(indice_usuario, n_recomendacoes=5):
    # Obter índices dos filmes que o usuário avaliou
    filmes_avaliados = matriz_usuario_filme.iloc[indice_usuario]
    filmes_avaliados_idx = filmes_avaliados[filmes_avaliados > 0].index.tolist()

    # Obter similaridade dos usuários
    indices_similares = similaridade_usuarios[indice_usuario].argsort()[::-1][1:n_recomendacoes + 1]

    # Obter recomendações baseadas em filmes avaliados por usuários similares
    recomendacoes = pd.Series(dtype='float64')
    for indice in indices_similares:
        filmes_similares = matriz_usuario_filme.iloc[indice]
        recomendacoes = pd.concat([recomendacoes, filmes_similares[filmes_similares > 0]])

    # Filtrar recomendações com base nos filmes já avaliados pelo usuário
    recomendacoes = recomendacoes[~recomendacoes.index.isin(filmes_avaliados_idx)]
    recomendacoes = recomendacoes.groupby(recomendacoes.index).mean().nlargest(n_recomendacoes)

    return recomendacoes.index.tolist()

In [44]:
# Função para recomendações híbridas
def recomendar_filmes_hibrido(indice_usuario, n_recomendacoes=5):
    # Recomendação por filtragem colaborativa
    recomendacoes_colaborativas = recomendar_filmes_usuario(indice_usuario, n_recomendacoes)

    # Obter os filmes recomendados
    indices_filmes_recomendados = [dados_filmes[dados_filmes['titulo_filme'] == filme].index[0] for filme in recomendacoes_colaborativas]

    # Obter similaridade dos filmes recomendados
    similaridade_filmes_recomendados = similaridade_filmes[indices_filmes_recomendados].mean(axis=0)

    # Obter recomendações baseadas na similaridade dos filmes recomendados
    recomendacoes_hibridas = dados_filmes['titulo_filme'][similaridade_filmes_recomendados.argsort()[::-1][:n_recomendacoes]]

    return recomendacoes_hibridas.tolist()

In [46]:
# Teste (usuário 1)
usuario_teste = 0
recomendacoes_colaborativas = recomendar_filmes_usuario(usuario_teste) # Base no comportamento de usuários semelhantes
recomendacoes_hibridas = recomendar_filmes_hibrido(usuario_teste)  # Base no comportamento de características dos filmes


print("Recomendações colaborativas:", recomendacoes_colaborativas)
print("Recomendações híbridas:", recomendacoes_hibridas)


Recomendações colaborativas: ['Chinatown (1974)', 'Close Shave, A (1995)', 'Dead Man (1995)', 'Ghost in the Shell (Kokaku kidotai) (1995)', 'Hamlet (1996)']
Recomendações híbridas: ['Further Gesture, A (1996)', 'Rough Magic (1995)', 'Four Rooms (1995)', "Brother's Kiss, A (1997)", 'Sliding Doors (1998)']
